In this notebook we create a Virtual Schema which accesses Json files on a public S3 bucket.
The data we're loading is a part of [Reuter 21578](https://paperswithcode.com/dataset/reuters-21578) dataset, which is a collection of documents with news articles. 

In [ ]:
%run ../utils/access_store_ui.ipynb
display(get_access_store_ui('../'))

As a first step we setting up the S3 Virtual Schema connector in the database.
You can find more information in the [s3-document-files-virtual-schema](https://github.com/exasol/s3-document-files-virtual-schema/) GitHub repo. 

In [ ]:
%run ./s3_vs_setup.ipynb

# Create Virtual Schema

To configure the virtual schema, we need the following:

1. Create an S3 Connection object: in what S3 bucket our data resides. Our data set is on the public S3 bucket, so no S3 credentials are needed. If your data is not open to the public, you'll need credentials to be stored in the Connection object.
2. Put EDML file into teh Bucket FS. EDML describes the mapping between Json structure and desired Virtual Schema tables and columns. In our case the mapping is simple, as every json file describes individual news article with the same set of columns. But EDML supports more complicated scenarios, like table references, foreign keys and missing columns. You can find more details [in the documentation](https://github.com/exasol/virtual-schema-common-document/blob/main/doc/user_guide/edml_user_guide.md)
3. Create Virtual Schema with S3 Connection object and EDML file in Bucket FS. After this step, new Virtual Schema tables are added to the Exasol, and every query from them performs S3 data read.

In [ ]:
import pyexasol
import pathlib
from exasol.nb_connector.connections import open_pyexasol_connection
from exasol.nb_connector import bfs_utils

In [ ]:
sql = """
CREATE CONNECTION S3_CONNECTION_VS
  TO ''
  USER ''
  IDENTIFIED BY '{
      "awsRegion": "eu-central-1", 
      "s3Bucket": "ai-lab-example-data-s3" 
  }';
"""

with open_pyexasol_connection(ai_lab_config) as conn:
    conn.execute(sql)

Our EDML extracts 3 fields from json files:
* `title`: as a varchar
* `id` (named as `NEWS_ID`): article identifier stored as a string
* `body`: text of the article up to 1024 characters long

In addition to those 3 fields, we have field `topics`, which is extracted as a reference table.
In the json data topics are represented as a list of tags associated with the article, for example:

```
    "topics": [
      "grain",
      "wheat",
      "corn",
      "barley",
      "oat",
      "sorghum"
    ],
```

To have convenient access to topics, we use `toTableMapping` EDML feature, which creates a second table `NEWS_TOPICS`, keeping list of topics and `NEWS_ID`:

```
      "topics": {
        "toTableMapping": {
          "mapping": {
            "toVarcharMapping": {
              "destinationName": "NAME"
            }
          }
        }
      },
```

In [ ]:
# put edml to bucket fs
bfs_bucket = open_bucketfs_connection(ai_lab_config)
bfs_path = bfs_utils.put_file(bfs_bucket, pathlib.Path("reuters-edml.json"))

In [ ]:
bfs_path.as_udf_path()

In [ ]:
sql = """
CREATE VIRTUAL SCHEMA {schema_name!i}_VS USING {schema_name!i}.S3_FILES_ADAPTER WITH
    CONNECTION_NAME = 'S3_CONNECTION_VS'
    MAPPING         = {map_file!s};
"""

with open_pyexasol_connection(ai_lab_config) as conn:
    conn.execute(sql, query_params={
        "schema_name": ai_lab_config.db_schema,
        "map_file": bfs_path.as_udf_path()
    })

# Query the data

Once we have our Virtual Schema created, new table `AI_LAB_VS.NEWS` is available for query. 
But you need to be aware of performance overhead. Virtal Schemas don't do any intermediate caching and every request to the Virtual Schema table performs several S3 data reads, which might be slow.

If you're going to access the data several times, it might make sense to create the new table and copy the data from VS table in a normal table. Here we don't do that for simplicity - we do just one query and that's fine. 

In [ ]:
sql = """
select title, news_id from {schema_name!i}_VS.NEWS
limit 10
"""

with open_pyexasol_connection(ai_lab_config) as conn:
    res = conn.execute(sql, query_params={
        "schema_name": ai_lab_config.db_schema
    })
    for r in res:
        print(r)

Besides the `NEWS` table, where each row is a news article, we have reference table with topics associated with every article.
In our EDML file, `toTableMapping` was used, which created a second table `NEWS_TOPICS`, containing flattened list of topics associated with `NEWS_ID`.

In [ ]:
sql = """
select *
from {schema_name!i}_VS.NEWS_TOPICS
limit 10
"""

with open_pyexasol_connection(ai_lab_config) as conn:
    res = conn.execute(sql, query_params={
        "schema_name": ai_lab_config.db_schema
    })
    for r in res:
        print(r)

So, we can can get most frequent topics for our dataset, for example

In [ ]:
sql = """
select NAME, count(*)
from {schema_name!i}_VS.NEWS_TOPICS
group by 1
order by 2 desc
limit 10
"""

with open_pyexasol_connection(ai_lab_config) as conn:
    res = conn.execute(sql, query_params={
        "schema_name": ai_lab_config.db_schema
    })
    for r in res:
        print(r)

Both tables `NEWS` and `NEWS_TOPICS` could be joined on `NEWS_ID` column:

In [ ]:
sql = """
select topics.name, news.title, news.news_id
from 
    {schema_name!i}_VS.NEWS as news,
    {schema_name!i}_VS.NEWS_TOPICS as topics
where news.NEWS_ID = topics.NEWS_NEWS_ID

limit 10
"""

with open_pyexasol_connection(ai_lab_config) as conn:
    res = conn.execute(sql, query_params={
        "schema_name": ai_lab_config.db_schema
    })
    for r in res:
        print(r)